In [4]:
import sys
import os
from pathlib import Path

# Get the root directory by moving up one level from "notebooks/"
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add root directory to sys.path
sys.path.append(root_dir)

# Now you can import using the module path
from pipeline.utils.duckdb_wrapper import DuckDBWrapper


In [5]:
# Initialize the DuckDBWrapper (in-memory DuckDB instance) You can connect directly to a DuckDB file by adding the path like con = DuckDBWrapper()
con = DuckDBWrapper()

In [ ]:
# Define the repository root and base path
repo_root = Path.cwd().resolve().parents[0]  # Adjust to locate the repo root
base_path = "data/opendata"

# Define table names for normal (non-partitioned) files
bulk_table_names = [

]

# Define table names for partitioned files
partitioned_table_names = [
    "mta_subway_hourly_ridership",
    "mta_subway_origin_destination_2023",
    "mta_subway_origin_destination_2024",
    "crime_nypd_arrests",
    "nyc_threeoneone_requests"
]

# Register normal files as views
con.bulk_register_data(
    repo_root=repo_root,
    base_path=base_path,
    table_names=bulk_table_names,
    wildcard="*.parquet",
    as_table=False,  # Default, registers as views
    show_tables=False
)

# Register partitioned files as views
con.bulk_register_partitioned_data(
    repo_root=repo_root,
    base_path=base_path,
    table_names=partitioned_table_names,
    wildcard="year=*/month=*/*.parquet",
    as_table=False,  # Default, registers as views
    show_tables=True  # Show the registered tables
)

In [7]:
query = f"""

SELECT * from mta_subway_hourly_ridership limit 20000

"""

result = con.run_query(query)

print(result)


shape: (20_000, 13)
┌──────────────┬───────────┬──────────────┬─────────────┬───┬───────────┬───────────┬───────┬──────┐
│ fare_class_c ┆ latitude  ┆ transit_mode ┆ station_com ┆ … ┆ transfers ┆ borough   ┆ month ┆ year │
│ ategory      ┆ ---       ┆ ---          ┆ plex_id     ┆   ┆ ---       ┆ ---       ┆ ---   ┆ ---  │
│ ---          ┆ f64       ┆ str          ┆ ---         ┆   ┆ f64       ┆ str       ┆ str   ┆ i64  │
│ str          ┆           ┆              ┆ str         ┆   ┆           ┆           ┆       ┆      │
╞══════════════╪═══════════╪══════════════╪═════════════╪═══╪═══════════╪═══════════╪═══════╪══════╡
│ OMNY - Full  ┆ 40.763973 ┆ subway       ┆ 224         ┆ … ┆ 0.0       ┆ Manhattan ┆ 03    ┆ 2022 │
│ Fare         ┆           ┆              ┆             ┆   ┆           ┆           ┆       ┆      │
│ OMNY - Full  ┆ 40.745907 ┆ subway       ┆ 165         ┆ … ┆ 2.0       ┆ Manhattan ┆ 03    ┆ 2022 │
│ Fare         ┆           ┆              ┆             ┆   ┆          

In [8]:
#If you want a better looking table, set show_results=True. I'd recomend capping the limit at about 50 rows
#T

query = f"""

SELECT * from mta_subway_hourly_ridership limit 10
"""

result = con.run_query(query,show_results=True)


In [9]:
#More complicated query

query = f"""

WITH weekly_ridership AS (
    SELECT 
        station_complex, 
        DATE_TRUNC('week', transit_timestamp) AS week_start,
        SUM(ridership) AS total_weekly_ridership,
        MIN(latitude) AS latitude,  -- Assuming latitude is the same for each station complex, use MIN() or MAX()
        MIN(longitude) AS longitude  -- Assuming longitude is the same for each station complex, use MIN() or MAX()
    FROM 
        mta_subway_hourly_ridership
    GROUP BY 
        station_complex, 
        DATE_TRUNC('week', transit_timestamp)
),
weekly_weather AS (
    SELECT 
        DATE_TRUNC('week', date) AS week_start,
        AVG(temperature_mean) AS avg_weekly_temperature,
        SUM(precipitation_sum) AS total_weekly_precipitation
    FROM 
        daily_weather_asset
    GROUP BY 
        DATE_TRUNC('week', date)
)
SELECT 
    wr.station_complex, 
    wr.week_start, 
    wr.total_weekly_ridership,
    wr.latitude,
    wr.longitude,
    ww.avg_weekly_temperature,
    ww.total_weekly_precipitation
FROM 
    weekly_ridership wr
LEFT JOIN 
    weekly_weather ww
ON 
    wr.week_start = ww.week_start
WHERE 
    wr.week_start < '2024-09-17'
ORDER BY 
    wr.station_complex, 
    wr.week_start
 
"""

result = con.run_query(query)

print(result)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

shape: (57_349, 7)
┌───────────────┬────────────┬───────────────┬───────────┬───────────┬──────────────┬──────────────┐
│ station_compl ┆ week_start ┆ total_weekly_ ┆ latitude  ┆ longitude ┆ avg_weekly_t ┆ total_weekly │
│ ex            ┆ ---        ┆ ridership     ┆ ---       ┆ ---       ┆ emperature   ┆ _precipitati │
│ ---           ┆ date       ┆ ---           ┆ f64       ┆ f64       ┆ ---          ┆ on           │
│ str           ┆            ┆ f64           ┆           ┆           ┆ f64          ┆ ---          │
│               ┆            ┆               ┆           ┆           ┆              ┆ f64          │
╞═══════════════╪════════════╪═══════════════╪═══════════╪═══════════╪══════════════╪══════════════╡
│ 1 Av (L)      ┆ 2022-02-28 ┆ 87331.0       ┆ 40.730953 ┆ -73.98163 ┆ 37.042857    ┆ 5.8          │
│ 1 Av (L)      ┆ 2022-03-07 ┆ 102571.0      ┆ 40.730953 ┆ -73.98163 ┆ 40.0         ┆ 34.6         │
│ 1 Av (L)      ┆ 2022-03-14 ┆ 100996.0      ┆ 40.730953 ┆ -73.98163 ┆ 4

In [10]:
# Show the tables registered
con.show_tables()


             Registered Tables              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Table Name                  ┃ Table Type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ daily_weather_asset         │ VIEW       │
├─────────────────────────────┼────────────┤
│ hourly_weather_asset        │ VIEW       │
├─────────────────────────────┼────────────┤
│ mta_daily_ridership         │ VIEW       │
├─────────────────────────────┼────────────┤
│ mta_operations_statement    │ VIEW       │
├─────────────────────────────┼────────────┤
│ mta_subway_hourly_ridership │ VIEW       │
└─────────────────────────────┴────────────┘

In [11]:
# Show the schema of a specific table
con.show_schema("mta_subway_hourly_ridership")

            Schema for             
   'mta_subway_hourly_ridership'   
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Column Name         ┃ Data Type ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ fare_class_category │ VARCHAR   │
│ latitude            │ DOUBLE    │
│ transit_mode        │ VARCHAR   │
│ station_complex_id  │ VARCHAR   │
│ longitude           │ DOUBLE    │
│ station_complex     │ VARCHAR   │
│ payment_method      │ VARCHAR   │
│ ridership           │ DOUBLE    │
│ transit_timestamp   │ TIMESTAMP │
│ transfers           │ DOUBLE    │
│ borough             │ VARCHAR   │
│ month               │ VARCHAR   │
│ year                │ BIGINT    │
└─────────────────────┴───────────┘

In [ ]:
query = f"""

SELECT * from mta_subway_hourly_ridership where year=2024 limit 100 

"""

result = con.run_query(query)

print(result)

repo_root = Path.cwd().resolve().parents[0]  # Adjust to locate the repo root
base_path = repo_root / "data/exports"
file_name = "mta_subway_hourly_ridership_data_sample"
file_type= "csv"
# Export the query result to CSV
con.export(result, file_type=file_type, base_path=base_path, file_name=file_name)